<a href="https://colab.research.google.com/github/krishan30/AiLERT-FYP/blob/main/DCL_Framework_with_huggingface_hyper_parameter_tuning_RoBERTa_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [ ]:
!pip install transformers
!pip install evaluate
!pip install accelerate -U
!pip install torchmetrics
!pip install optuna_dashboard
!pip install optuna
!pip install -U "neptune[optuna]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.5/481.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,WeightedRandomSampler
from transformers import AutoModel,AutoTokenizer,AdamW,get_scheduler
from tqdm import tqdm
import evaluate
from datasets import load_dataset
from tqdm.auto import tqdm
import numpy as np
import random
from torchmetrics.classification import BinaryAccuracy
import optuna
from google.colab import userdata
import neptune
import uuid
import neptune.integrations.optuna as npt_utils
import  hashlib
import time

## Setting up the GPU or CPU

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["NEPTUNE_API_TOKEN"] = userdata.get('NEPTUNE_API_TOKEN')
os.environ["NEPTUNE_PROJECT"] = userdata.get('NEPTUNE_PROJECT')
device = torch.device("cuda:0"  if torch.cuda.is_available() else "cpu")

## Hyperparameter Values

In [ ]:
#@title #Hyperparameter Values
BATCH_SIZE = 144 # @param {type:"integer"}
EPOCHS = 2 # @param {type:"integer"}
DROPOUT =0.5# @param {type:"number"}
ALPHA = 0.4601079528693497 # @param {type:"number"}
GAMMA = 4.932651675660353 # @param {type:"number"}
TEMP_1 = 0.14630528484684996 # @param {type:"number"}
TEMP_2 =  0.07242998464641397# @param {type:"number"}
LAMBDA = 1
SEED = 1234 # @param {type:"integer"}
PADDING_MAX_LENGTH = 45 # @param {type:"integer"}


## Setting Random Seed for Reproducibility

In [ ]:
def setup_seed(seed:int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    #torch.random.manual_seed(seed)
    torch.backends.cudnn.deterministic = True



In [ ]:
setup_seed(SEED)

In [ ]:
# @title Bert Model
bert_model_name = "roberta-base" # @param {type:"string"}

## Loading Train,Dev,Test Datasets

In [ ]:
dataset = load_dataset("krishan-CSE/HatEval_New")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Loading the Tokernizer for the Transformer Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer.batch_encode_plus(examples["text"], padding='max_length',max_length=PADDING_MAX_LENGTH,add_special_tokens=True,truncation=True)

## Tokenize the Dataset

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 8982
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 998
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 3000
    })
})

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/8982 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 8982
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 998
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 3000
    })
})

## Remove Unwanted Coloumns

In [ ]:
tokenized_datasets=tokenized_datasets.remove_columns(['text'])

## Format the coloumns

In [ ]:
tokenized_datasets=tokenized_datasets.with_format("torch")

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 8982
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 998
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3000
    })
})

## Creating DataLoaders for Train,Dev,Test Datasets

## Define the Bert Architecture



In [ ]:
class DCLArchitecture(nn.Module):
    def __init__(self,dropout:float,bert_model_name:str='bert-base-cased'):
        super(DCLArchitecture, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.dim = 768
        self.dense = nn.Linear(self.dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self,batch_tokenized, if_train=False):
        input_ids = batch_tokenized['input_ids']
        attention_mask = batch_tokenized['attention_mask']
        bert_output = self.bert(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        bert_cls_hidden_state = bert_output[1]
        torch.cuda.empty_cache()

        if if_train:
            bert_cls_hidden_state_aug = self.dropout(bert_cls_hidden_state)
            bert_cls_hidden_state = torch.cat((bert_cls_hidden_state, bert_cls_hidden_state_aug), dim=1).reshape(-1, self.dim)
        else:
            bert_cls_hidden_state = self.dropout(bert_cls_hidden_state)

        linear_output = self.dense(bert_cls_hidden_state)
        linear_output = linear_output.squeeze(1)

        return bert_cls_hidden_state, linear_output

## Define Focal Loss

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha:float=0.4, gamma:float=2, size_average:bool=True):
        super(FocalLoss, self).__init__()
        self.alpha = torch.tensor(alpha)
        self.gamma = gamma
        self.size_average = size_average

    def forward(self, pred, target):

        device = target.device
        self.alpha = self.alpha.to(device)

        pred = nn.Sigmoid()(pred)
        pred = pred.view(-1, 1)
        target = target.view(-1, 1)
        pred = torch.cat((1-pred, pred), dim=1)

        class_mask = torch.zeros(pred.shape[0], pred.shape[1]).to(device)
        class_mask.scatter_(1, target.view(-1, 1).long(), 1.)
        probs = (pred * class_mask).sum(dim=1).view(-1, 1)
        probs = probs.clamp(min=0.0001, max=1.0)

        log_p = probs.log()
        alpha = torch.ones(pred.shape[0], pred.shape[1]).to(device)
        alpha[:, 0] = alpha[:, 0] * (1 - self.alpha)
        alpha[:, 1] = alpha[:, 1] * self.alpha
        alpha = (alpha * class_mask).sum(dim=1).view(-1, 1)

        batch_loss = -alpha * (torch.pow((1 - probs), self.gamma)) * log_p

        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()

        return loss

## Define Unsupervised Contrastive loss

In [ ]:
def simcse_loss(batch_emb,temp_1:float):
    batch_size = batch_emb.size(0)
    y_true = torch.cat([torch.arange(1, batch_size, step=2, dtype=torch.long).unsqueeze(1),
                        torch.arange(0, batch_size, step=2, dtype=torch.long).unsqueeze(1)],
                       dim=1).reshape([batch_size,]).to(device)
    norm_emb = F.normalize(batch_emb, dim=1, p=2)
    sim_score = torch.matmul(norm_emb, norm_emb.transpose(0,1))
    sim_score = sim_score - (torch.eye(batch_size) * 1e12).to(device)
    sim_score = sim_score / temp_1
    loss_func = nn.CrossEntropyLoss()
    loss = loss_func(sim_score, y_true)

    return loss


## Define Supervised Contrastive loss

In [ ]:
def sup_simcse_loss(batch_emb, label,temp_2:float):
    n = batch_emb.size(0)

    similarity_matrix = F.cosine_similarity(batch_emb.unsqueeze(1), batch_emb.unsqueeze(0), dim=2)
    mask = torch.ones_like(similarity_matrix) * (label.expand(n, n).eq(label.expand(n, n).t()))

    mask_no_sim = torch.ones_like(mask) - mask
    mask_dui_jiao_0 = ((torch.ones(n,n) - torch.eye(n,n)) * 1e12).to(device)
    similarity_matrix = torch.exp(similarity_matrix/temp_2)
    similarity_matrix = similarity_matrix * mask_dui_jiao_0

    sim = mask*similarity_matrix
    no_sim = similarity_matrix - sim
    no_sim_sum = torch.sum(no_sim , dim=1)
    no_sim_sum_expend = no_sim_sum.repeat(n, 1).T

    sim_sum  = sim + no_sim_sum_expend
    loss = torch.div(sim , sim_sum)
    loss = mask_no_sim + loss + (torch.eye(n, n)/1e12).to(device)
    loss = -torch.log(loss)
    loss = torch.sum(torch.sum(loss, dim=1))/(2*n)
    return loss

## Fine Tuning the Model

In [ ]:
# @title Input study_name
study_name = "DCL_Experiments_2" # @param {type:"string"}



In [ ]:
optuna_storage_location="content/drive/MyDrive/FYP/Hyper_Parameter_Tuning/RoBERTa_Model/optimization_study.db"
study_id=str(hashlib.md5(str(time.time()).encode()).hexdigest()[:8])

###Setting up Neptuna

In [ ]:
sweep_id = uuid.uuid1()

In [ ]:
run_study_level = neptune.init_run(capture_stdout=True,capture_stderr=True,capture_traceback=True, capture_hardware_metrics=True,tags=["huggingface", "hyper parameter tuning", "RoBERTa"],name=(study_name+'_'+study_id))

https://app.neptune.ai/FYP-DCL/DCL-Framework/e/DCLFRAM-81


In [ ]:
run_study_level["sweep-id"] = str(sweep_id)

In [ ]:
run_study_level["sys/tags"].add("study-level")

## Format the Labels

In [ ]:
def copy_label(labels):
    labels = labels.unsqueeze(1)
    labels = torch.cat((labels, labels), dim=1).reshape(-1, 1).squeeze(1)
    return labels

### Train the Model

In [ ]:
def train_model(model, train_dataloader,optimizer,criteon,device,temp_1:float,temp_2:float,lamda:float,run_trial_level):
    accuracy_metric = BinaryAccuracy()
    accuracy_metric.to(device)
    progress_bar = tqdm(range(len(train_dataloader)))
    model.train()
    total_train_loss = 0.0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = copy_label(batch["labels"])
        emb, pred = model(batch, if_train=True)
        loss = criteon(pred, labels.float())
        loss_sim = simcse_loss(emb,temp_1=TEMP_1)
        loss_supsim = sup_simcse_loss(emb, labels,temp_2=TEMP_2)
        total_train_loss += loss.item()
        run_trial_level["train_batch/loss"].append(loss.item())
        run_trial_level["train_batch/Unsupervised_Contrastive_loss"].append(loss_sim)
        run_trial_level["train_batch/supervised_Contrastive_loss"].append(loss_supsim)
        pred = torch.sigmoid(pred)
        batch_level_accuracy=accuracy_metric(pred,labels)
        run_trial_level["train_batch/accuracy"].append(batch_level_accuracy.item())
        loss = loss + lamda*(loss_sim + loss_supsim)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progress_bar.update(1)
    average_epoch_train_loss = total_train_loss / len(train_dataloader)  # Compute average epoch loss
    train_accuracy = accuracy_metric.compute()
    accuracy_metric.reset()
    return average_epoch_train_loss,train_accuracy


### Evaluate the Model

In [ ]:
def evaluate_model(model, dev_dataloader,criteon, device):
    accuracy_metric = BinaryAccuracy()
    accuracy_metric.to(device)
    model.eval()

    total_dev_loss = 0.0
    for batch in dev_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["labels"]
        with torch.no_grad():
            emb, pred = model(batch , False)
            loss = criteon(pred, labels.float())
        total_dev_loss += loss.item()
        pred = torch.sigmoid(pred)
        accuracy_metric(pred,labels)

    average_epoch_dev_loss = total_dev_loss / len(dev_dataloader)  # Compute average epoch loss
    dev_accuracy = accuracy_metric.compute()
    accuracy_metric.reset()
    return average_epoch_dev_loss,dev_accuracy

### Train & Evaluate Model

## Hyperparameter Tuning

In [ ]:
storage = optuna.storages.RDBStorage("sqlite:////"+optuna_storage_location)

In [ ]:
def define_model(bert_model_name,trial):
    DROPOUT = 0.5
    model = DCLArchitecture(bert_model_name=bert_model_name,dropout=DROPOUT)
    model.to(device)
    return model

In [ ]:
def tokenized_batch_data(trial,tokenized_datasets):
    BATCH_SIZE = trial.suggest_int("batch_size",100,220)
    train_dataloader=DataLoader(tokenized_datasets["train"], batch_size=BATCH_SIZE , shuffle=True)
    dev_dataloader=DataLoader(tokenized_datasets["validation"], batch_size=BATCH_SIZE , shuffle=True)
    return train_dataloader,dev_dataloader,BATCH_SIZE


In [ ]:
def objective(trial):
    # Define the hyperparameters to optimize

    EPOCHS = trial.suggest_int("epochs", 1,2)
    ALPHA= trial.suggest_float("alpha", 0.1, 0.5)
    GAMMA = trial.suggest_float("gamma", 1.0, 5)
    TEMP_1 = trial.suggest_float("temp_1", 0.05, 0.15)
    TEMP_2 = trial.suggest_float("temp_2", 0.01,0.1)
    train_dataloader,dev_dataloader,BATCH_SIZE =tokenized_batch_data(trial,tokenized_datasets)
    model = define_model(bert_model_name,trial)
    optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-4)
    criteon = FocalLoss(ALPHA, GAMMA)
    param = {
        "EPOCHS": EPOCHS,
        "ALPHA": ALPHA,
        "GAMMA": GAMMA,
        "TEMP_1": TEMP_1,
        "TEMP_2":TEMP_2,
        "BATCH_SIZE":BATCH_SIZE,
        "lr":1e-4,
        "DROPOUT":0.5,
        "LAMBDA":1,
        "SEED":SEED


    }
     # Create a trial-level run
    run_trial_level = neptune.init_run(capture_stdout=True,capture_stderr=True,capture_traceback=True, capture_hardware_metrics=True,
    name=study_name+str(study_id)+"_"+str(trial.number),tags=["hyper parameter tuning",(study_name+'_'+study_id),str(trial.number+1)])

    # Log sweep ID to trial-level run
    run_trial_level["sys/tags"].add("trial-level")
    run_trial_level["sweep-id"] = str(sweep_id)

    # Log parameters of a trial-level run
    run_trial_level["parameters"] = param

    for epoch in range(EPOCHS):
        average_epoch_train_loss,train_accuracy = train_model(model, train_dataloader,optimizer,criteon,device,TEMP_1,TEMP_2,LAMBDA,run_trial_level)
        average_epoch_dev_loss,dev_accuracy = evaluate_model(model, dev_dataloader,criteon, device)

        trial.report(dev_accuracy, epoch)

        # Log score of a trial-level Run
        run_trial_level["train/epoch/loss"].append(average_epoch_train_loss)
        run_trial_level["train/epoch/accuracy"].append(train_accuracy)
        run_trial_level["evaluation/epoch/loss"].append(average_epoch_dev_loss)
        run_trial_level["evaluation/epoch/accuracy"].append(dev_accuracy)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            run_trial_level.stop()
            raise optuna.exceptions.TrialPruned()
    run_trial_level["train/loss"]=average_epoch_train_loss
    run_trial_level["train/accuracy"]=train_accuracy
    run_trial_level["evaluation/loss"]=average_epoch_dev_loss
    run_trial_level["evaluation/accuracy"]=dev_accuracy
    torch.save(model.state_dict(),'model_'+str(trial.number)+'.pt')
    torch.save(criteon.state_dict(),'criteon_'+str(trial.number)+'.pt')
    run_trial_level["model_checkpoints/model"].upload('model_'+str(trial.number)+'.pt')
    run_trial_level["model_checkpoints/criteon"].upload('criteon_'+str(trial.number)+'.pt')
    # Stop trial-level Run
    run_trial_level.stop()
    print("Train_accuracy",train_accuracy)

    return dev_accuracy


In [ ]:
neptune_callback = npt_utils.NeptuneCallback(run_study_level)

In [ ]:
study = optuna.create_study(study_name=study_name,storage=storage,load_if_exists=True,direction='maximize')
study.optimize(objective, n_trials=50,callbacks=[neptune_callback])

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2023-12-08 16:43:20,861] A new study created in RDB with name: DCL-Experiments_2


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


https://app.neptune.ai/FYP-DCL/DCL-Framework/e/DCLFRAM-82


  0%|          | 0/88 [00:00<?, ?it/s]

...

https://app.neptune.ai/FYP-DCL/DCL-Framework/e/DCLFRAM-131


  0%|          | 0/45 [00:00<?, ?it/s]

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 9 operations to synchronize with Neptune. Do not kill this process.
All 9 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/FYP-DCL/DCL-Framework/e/DCLFRAM-131/metadata


[I 2023-12-08 18:28:47,730] Trial 49 pruned. 


Accuracy: 0.757515013217926
Best hyperparameters: {'epochs': 2, 'alpha': 0.4429593504449712, 'gamma': 1.8775783926917704, 'temp_1': 0.05137844981376246, 'temp_2': 0.07135548353185929, 'batch_size': 108}


In [ ]:
run_study_level.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 13 operations to synchronize with Neptune. Do not kill this process.
All 13 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/FYP-DCL/DCL-Framework/e/DCLFRAM-81/metadata


In [ ]:
trial 25
Accuracy: 0.757515013217926
Best hyperparameters: {'epochs': 2, 'alpha': 0.4429593504449712, 'gamma': 1.8775783926917704, 'temp_1': 0.05137844981376246, 'temp_2': 0.07135548353185929, 'batch_size': 108}